### Gelest Website

In [1]:
import PyPDF2
from urllib.request import urlopen
import requests
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import itertools
from bs4 import BeautifulSoup
from rdkit import Chem
from rdkit.Chem import Draw

## Sigam Aldrich

In [5]:
url = 'https://www.sigmaaldrich.com/catalog/search?interface=All&N=0+16245265&mode=partialmax&focus=product&lang=en&region=US'
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
products = soup.find_all("div", {"class" : "productContainer-inner-content"})

In [49]:
num = products[0].find_all('li', {'class' : 'productNumberValue'})
link = num[0].find_all('a', href=True)#ls = num.find_all('a', href=True)
link[0]['href']

'/catalog/product/aldrich/766917?lang=en®ion=US'

In [54]:
sigma= 'https://www.sigmaaldrich.com'
for card in products:
    #find link to info page
    subset = card.find_all('li', {'class' : 'productNumberValue'})
    link = subset[0].find_all('a', href=True)#ls = num.find_all('a', href=True)
    ending = link[0]['href']
    new_url = sigma+ending
    
    #get response
    response = requests.get(new_url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    #search for flashpoint
    flashpoint = soup.find_all('div', {'id' : 'Flash Point(C)'})
    print(flashpoint)
    if flashpoint != []: 
        print(flashpoint[1])
    
    #products[1]#.find("li", {"class" : "productNumberValue"})

[<div class="safetyLeft" id="Flash Point(C)">
         Flash Point(C) 
        </div>, <div class="safetyRight" id="Flash Point(C)">
         0 °C
        </div>]
<div class="safetyRight" id="Flash Point(C)">
         0 °C
        </div>
[]
[<div class="safetyLeft" id="Flash Point(C)">
         Flash Point(C) 
        </div>, <div class="safetyRight" id="Flash Point(C)">
         77 °C
        </div>]
<div class="safetyRight" id="Flash Point(C)">
         77 °C
        </div>
[<div class="safetyLeft" id="Flash Point(C)">
         Flash Point(C) 
        </div>, <div class="safetyRight" id="Flash Point(C)">
         41 °C
        </div>]
<div class="safetyRight" id="Flash Point(C)">
         41 °C
        </div>
[<div class="safetyLeft" id="Flash Point(C)">
         Flash Point(C) 
        </div>, <div class="safetyRight" id="Flash Point(C)">
         91 °C
        </div>]
<div class="safetyRight" id="Flash Point(C)">
         91 °C
        </div>
[<div class="safetyLeft" id="Flash Poin

In [3]:
url = 'https://www.gelest.com/product-lines/silanes/?pl_page=1&perpage=50&product-code-filter=SIA'
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
products = soup.find("div", {"class": re.compile("^(first border-box col-xs-24)")})
href = soup.find("a", {"class": "next"})


In [5]:
#helper methods
def getFlashPoint(pdfUrl):
    flashp = -1
    response = urlopen(pdfUrl)
    file = open("document.pdf", 'wb')
    file.write(response.read())
    file.close()
    pdf_file = open('document.pdf', 'rb')
        
    read_pdf = PyPDF2.PdfFileReader(pdf_file)
    
    #number_of_pages = read_pdf.getNumPages()
    for p in range(read_pdf.getNumPages()):
        page = read_pdf.getPage(p)
        text = page.extractText()
        
        text = text.replace('\n', ' ').replace('\r', '')
        fp = re.search('Flash point(.+?)C', text)
        #print(fp)
        if fp != None:
            fp = re.search('Flash point(.+?)C', text).group(0)
            flashPoint = re.findall('\d+', fp)
            #print(int(flashPoint[0]))
            flashp = int(flashPoint[0])
            break
    
    pdf_file.close()
    
    #print(flashp)    
    return flashp

def removeJunk(compound):
    compound = compound.split(',')
    return compound[0]

def scrap(url, frame, pageNumber):
    num = 1
    while num <= pageNumber:
        print("-------------------Page"+str(num)+"------------------------------------------")
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        products = soup.find("div", {"class": re.compile("^(first border-box col-xs-24)")})
        contents = soup.find_all("div", {"class": "wc-product-meta"})
        imgs = soup.find_all("img", {"class": "img-responsive center-block wp-post-image"})

        for img, data in zip(imgs, contents):

            productCode = data.find("span", {"class" : "code"}).text
            sds = soup.find("a", {"href": "https://s3.amazonaws.com/gelest/sds/"+productCode+"_GHS+US_English+US.pdf"})
            casNo = data.find("span", {"class" : "cas-number"})

            if casNo == None:
                casNo = ''
            else:
                casNo = casNo.text

            compound = data.find("p", {"class" : "title"}).text.strip('\n\t')
            compound = removeJunk(compound)
            smilesUrl = "https://opsin.ch.cam.ac.uk/opsin/"+compound+".smi"
            #print(smilesUrl)
            requestSmiles = requests.get(smilesUrl)

            if requestSmiles.status_code == 400 or requestSmiles.status_code == 404:
                print("no smiles")
            else:
                try:
                    if sds != None: #if sds['href'] != None:
                        #print(sds['href'])
                        if sds == None:
                            flashPoint = -1
                        else:
                            print(requestSmiles.text)
                            flashPoint = getFlashPoint(sds['href'])
                        frame = frame.append({'productCode' : productCode,
                                 'casNum' : casNo,
                                 'compound' : compound,
                                  'smiles' : requestSmiles.text,
                                  'flashPoint' : flashPoint,
                                 'img' : img['src'],
                                 'sds' : sds['href']}, ignore_index=True)
                except:
                    print('could not scrap pdf')
                    #print(compound, requestSmiles.text, flashPoint)
                    #print(sds["href"])

        num+= 1
        href = soup.find("a", {"class": "next"})
        url = 'https://www.gelest.com'+href['href']
        print(url)
    return frame

In [6]:
num = 3
frame = pd.DataFrame(columns=("productCode", "casNum", "compound", "smiles", "flashPoint", "img", "sds")) #frame for scraped data
url = 'https://www.gelest.com/product-lines/silanes/?pl_page=1&perpage=50&product-code-filter=SIA'
silanes = scrap(url, frame, num)
silanes.to_csv('silanes.csv', encoding='utf-8',index=False)

-------------------Page1------------------------------------------
C(C)(=O)NCCC[Si](OC)(OC)OC
C(C)(=O)OCC[Si](Cl)(C)C
C(C)(=O)OCC[Si](Cl)(Cl)C
C(C)(=O)OCC[Si](Cl)(Cl)Cl
C(C)(=O)OCC[Si](OCC)(OCC)OCC
C(C)(=O)OCC[Si](OC)(OC)OC
C(C)(=O)OCC[Si](N(C)C)(N(C)C)N(C)C
C(C)(=O)O[Si]1(O[Si](O[Si](O[Si](O1)(C)C)(C)C)(C)C)C
C(C)(=O)OC[Si](OCC)(OCC)OCC
C(C)(=O)OC[Si](OC)(OC)OC
could not scrap pdf
C(C)(=O)OC[Si](C)(C)C
C(C)(=O)OC1=CC=C(CC[Si](Cl)(Cl)Cl)C=C1
no smiles
no smiles
C(C)(=O)OCCC[Si](O[Si](C)(C)C)(O[Si](C)(C)C)C
C(C)(=O)OCCC[Si](Cl)(Cl)C
C(C)(=O)OCCC[Si](OC)(OC)C
C(C)(=O)OCCC[Si](OC)(OC)OC
C(C)(=O)O[Si](C)(C)C
C(C)(=O)OCCCCCCCCCCC[Si](Cl)(Cl)Cl
C(C)(=O)OCCCCCCCCCCC[Si](OCC)(OCC)OCC
C(C)(=O)NCC(=O)NCCC[Si](OC)(OC)OC
C(C)(=O)N1[C@@H](CC(C1)O)C(=O)OCCC[Si](OCC)(OCC)OCC
C(C)(=O)N[C@@H](CC(C)C)C(=O)NCCC[Si](OCC)(OCC)OCC
C(C)(=O)[Si](C)(C)C
C(C=C)(=O)NCCC[Si](OC)(OC)OC
C(C=C)(=O)NCCC[Si](O[Si](C)(C)C)(O[Si](C)(C)C)O[Si](C)(C)C
C(C=C)(=O)OCCO[Si](C)(C)C
C(C=C)(=O)OCC(CNCCC[Si](OCC)(OCC)OCC)O
C(C=C)

In [9]:
print('scraped silanes '+str(len(silanes)))
silanes #.drop_duplicates(subset='smiles')

scraped silanes 105


,productCode,casNum,compound,smiles,flashPoint,img,sds
0,SIA0006.0,57757-66-1,(3-ACETAMIDOPROPYL)TRIMETHOXYSILANE,C(C)(=O)NCCC[Si](OC)(OC)OC,65,https://www.gelest.com:443/wp-content/uploads/...,https://s3.amazonaws.com/gelest/sds/SIA0006.0_...
1,SIA0010.0,18306-45-1,ACETOXYETHYLDIMETHYLCHLOROSILANE,C(C)(=O)OCC[Si](Cl)(C)C,63,https://www.gelest.com:443/wp-content/uploads/...,https://s3.amazonaws.com/gelest/sds/SIA0010.0_...
2,SIA0015.0,18163-34-3,ACETOXYETHYLMETHYLDICHLOROSILANE,C(C)(=O)OCC[Si](Cl)(Cl)C,65,https://www.gelest.com:443/wp-content/uploads/...,https://s3.amazonaws.com/gelest/sds/SIA0015.0_...
3,SIA0020.0,18204-80-3,ACETOXYETHYLTRICHLOROSILANE,C(C)(=O)OCC[Si](Cl)(Cl)Cl,82,https://www.gelest.com:443/wp-content/uploads/...,https://s3.amazonaws.com/gelest/sds/SIA0020.0_...
4,SIA0025.0,22538-45-0,ACETOXYETHYLTRIETHOXYSILANE,C(C)(=O)OCC[Si](OCC)(OCC)OCC,99,https://www.gelest.com:443/wp-content/uploads/...,https://s3.amazonaws.com/gelest/sds/SIA0025.0_...
5,SIA0030.0,72878-29-6,ACETOXYETHYLTRIMETHOXYSILANE,C(C)(=O)OCC[Si](OC)(OC)OC,60,https://www.gelest.com:443/wp-content/uploads/...,https://s3.amazonaws.com/gelest/sds/SIA0030.0_...
6,SIA0035.0,1356113-09-1,ACETOXYETHYLTRIS(DIMETHYLAMINO)SILANE,C(C)(=O)OCC[Si](N(C)C)(N(C)C)N(C)C,40,https://www.gelest.com:443/wp-content/uploads/...,https://s3.amazonaws.com/gelest/sds/SIA0035.0_...
7,SIA0038.0,14697-86-0,ACETOXYHEPTAMETHYLCYCLOTETRASILOXANE,C(C)(=O)O[Si]1(O[Si](O[Si](O[Si](O1)(C)C)(C)C)...,65,https://www.gelest.com:443/wp-content/uploads/...,https://s3.amazonaws.com/gelest/sds/SIA0038.0_...
8,SIA0050.0,5630-83-1,ACETOXYMETHYLTRIETHOXYSILANE,C(C)(=O)OC[Si](OCC)(OCC)OCC,60,https://www.gelest.com:443/wp-content/uploads/...,https://s3.amazonaws.com/gelest/sds/SIA0050.0_...
9,SIA0060.0,2917-65-9,ACETOXYMETHYLTRIMETHYLSILANE,C(C)(=O)OC[Si](C)(C)C,20,https://www.gelest.com:443/wp-content/uploads/...,https://s3.amazonaws.com/gelest/sds/SIA0060.0_...


In [14]:
silanes = silanes.drop_duplicates(subset='smiles')
#print('scraped silanes after removing duplicate smiles'+str(len(silanes)))

In [15]:
print('scraped silanes after removing duplicate smiles '+str(len(silanes)))

scraped silanes after removing duplicate smiles102


In [16]:
silanes.to_csv('silanes.csv', encoding='utf-8',index=False)

In [17]:
num = 16
frame = pd.DataFrame(columns=("productCode", "casNum", "compound", "smiles", "flashPoint", "img", "sds")) #frame for scraped data
url = 'https://www.gelest.com/product-lines/metal-organics/?pl_page=1&perpage=50&product-code-filter='
silanes = scrap(url, frame, num)
silanes.to_csv('metalOrganics.csv', encoding='utf-8',index=False)

-------------------Page1------------------------------------------
no smiles
[O-]CCCC.[Al+3].[O-]CCCC.[O-]CCCC
no smiles
no smiles
no smiles
no smiles
CC(C)(C)[O-].[Al+3].CC(C)(C)[O-].CC(C)(C)[O-]
ClC1=C2C=CC(=NC2=C(C=C1)O)C(=O)[O-].[Al+3].ClC1=C2C=CC(=NC2=C(C=C1)O)C(=O)[O-].ClC1=C2C=CC(=NC2=C(C=C1)O)C(=O)[O-]
no smiles
C(C)OC(CC(=O)C)=O.CC([O-])C.CC([O-])C.[Al+2]
[O-]CC.[Al+3].[O-]CC.[O-]CC
could not scrap pdf
no smiles
no smiles
OC=1C=CC=C2C=CC(=NC12)C(=O)[O-].[Al+3].OC=1C=CC=C2C=CC(=NC12)C(=O)[O-].OC=1C=CC=C2C=CC(=NC12)C(=O)[O-]
CC([O-])C.[Al+3].CC([O-])C.CC([O-])C
no smiles
N(=O)N(O)C1=CC=CC=C1.[Al]
CC([O-])C.CC([O-])C.C(CCCCCCCC=CCCCCCCCC)CC(CC(=O)[O-])=O.[Al+3]
no smiles
no smiles
no smiles
no smiles
[O-]CCCC.[Sb+3].[O-]CCCC.[O-]CCCC
[O-]CC.[Sb+3].[O-]CC.[O-]CC
[O-]CC.[O-]CC.[O-]CC.[As+3]
CC([O-])C.[Ba+2].CC([O-])C
CC([O-])C.[Ba+2].CC([O-])C
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
CC([O-])C.[Bi+3].CC([O-])C.CC([O-])C
no smiles
no smiles
C(C=C)OCC=C.[B]
[O-]CCC

no smiles
CN(C([S-])=S)C.[Zn+2].CN(C([S-])=S)C
OC=1C=CC=C2C=CC(=NC12)C(=O)[O-].[Zn+2].OC=1C=CC=C2C=CC(=NC12)C(=O)[O-]
COC([O-])C.[Zn+2].COC([O-])C
no smiles
no smiles
no smiles
no smiles
[O-]CCCC.[Zr+4].[O-]CCCC.[O-]CCCC.[O-]CCCC
CC(C)(C)[O-].[Zr+4].CC(C)(C)[O-].CC(C)(C)[O-].CC(C)(C)[O-]
https://www.gelest.com/product-lines/metal-organics/?pl_page=8&perpage=50&product-code-filter=
-------------------Page8------------------------------------------
no smiles
no smiles
no smiles
[O-]CC.[Zr+4].[O-]CC.[O-]CC.[O-]CC
no smiles
no smiles
no smiles
CC([O-])C.[Zr+4].CC([O-])C.CC([O-])C.CC([O-])C
no smiles
no smiles
no smiles
[O-]CCC.[Zr+4].[O-]CCC.[O-]CCC.[O-]CCC
no smiles
no smiles
C(C)(=O)[O-].[Al+3].C(C)(=O)[O-].C(C)(=O)[O-]
C(C=C)(=O)[O-].[Al+3].C(C=C)(=O)[O-].C(C=C)(=O)[O-]
C(=O)[O-].[Al+3].C(=O)[O-].C(=O)[O-]
C(C(=C)C)(=O)[O-].[Al+3].C(C(=C)C)(=O)[O-].C(C(=C)C)(=O)[O-]
could not scrap pdf
no smiles
FC(C(=O)[O-])(F)F.[Al+3].FC(C(=O)[O-])(F)F.FC(C(=O)[O-])(F)F
FC(S(=O)(=O)[O-])(F)F.[Al+3].FC

no smiles
C(C)[Ga](CC)CC
C[Ga](C)C
no smiles
CN(C)[Ga](N(C)C)N(C)C
[Au]Cl.C(C)P(CC)CC
C(C)[N-]CC.[Hf+4].C(C)[N-]CC.C(C)[N-]CC.C(C)[N-]CC
C[N-]C.[Hf+4].C[N-]C.C[N-]C.C[N-]C
C(C)[N-]C.C(C)[N-]C.C(C)[N-]C.C(C)[N-]C.[Hf+4]
C(C)[N-]C.C(C)[N-]C.C(C)[N-]C.C(C)[N-]C.[Hf+4]
no smiles
C[In](C)C
[Cl-].C1=CC=CCCCC1.[Ir+]
C(CCC)[Li]
C[Li]
could not scrap pdf
no smiles
no smiles
no smiles
https://www.gelest.com/product-lines/metal-organics/?pl_page=14&perpage=50&product-code-filter=
-------------------Page14------------------------------------------
C(C)(C)(C)P(Cl)Cl
F[P-](F)(F)(F)(F)F.C(CCC)[N+]1=CN(C=C1)C
C1(CCCCC1)PC1CCCCC1
C(C)OP(OCC)(=O)CCCCCCCCCCCCCCCCCC
C1(=CC=CC=C1)P(Cl)C1=CC=CC=C1
C(C)OP(OCC)[O-]
C1(=CC=CC=C1)PC1=CC=CC=C1
C1(=CC=CC=C1)PC1=CC=CC=C1
C1(=CC=CC=C1)P(=O)(C1=CC=CC=C1)N=[N+]=[N-]
[OH-].C(CCC)[P+](CCCC)(CCCC)CCCC
could not scrap pdf
C(CCCCCCCCCCC)P(O)(O)=O
C(CCCCCCCCCCCCCCC)P(O)(O)=O
C(CCCCC)P(O)(O)=O
C(CCCCCCC)P(O)(O)=O
C(CCCCCCCCCCCCCCCCC)P(O)(O)=O
[Cl-].C(CCC)[P+](CCCC)(CCCC)CCC

In [18]:
num = 3
frame = pd.DataFrame(columns=("productCode", "casNum", "compound", "smiles", "flashPoint", "img", "sds")) #frame for scraped data
url = 'https://www.gelest.com/product-lines/germanium-compounds/?pl_page=1&perpage=50'
germanium = scrap(url, frame, num)
germanium.to_csv('germanium.csv', encoding='utf-8',index=False)

-------------------Page1------------------------------------------
C(C=C)[Ge](Cl)(Cl)Cl
C(C=C)[Ge](CC)(CC)CC
C(C=C)[Ge](C)(C)C
no smiles
C(C1=CC=CC=C1)[Ge](Cl)(Cl)Cl
C[Si](C)(C)N([Si](C)(C)C)[Ge]N([Si](C)(C)C)[Si](C)(C)C
ClC[Ge](C)(C)CCl
C(C)(C)(C)[Ge](Cl)(C)C
C(CCC)[GeH3]
C(C)(C)(C)[GeH3]
C(CCC)[Ge](Cl)(Cl)Cl
C(C)(C)(C)[Ge](Cl)(Cl)Cl
no smiles
no smiles
C(=O)(O)CC[Ge](Cl)(Cl)Cl
ClC[Ge](C)(C)C
ClCCC[Ge](Cl)(Cl)Cl
could not scrap pdf
C(CCC)[Ge](OC(C)=O)(OC(C)=O)CCCC
C(CCC)[Ge](Cl)(Cl)CCCC
ClC(Cl)[Ge](C)(C)C
C(C)[Ge](Cl)(Cl)CC
C(C)[Ge](OCC)(OCC)CC
C(C)[GeH2]CC
CN(C)[Ge](C)(C)C
C[Ge](Cl)(Cl)C
C1(=CC=CC=C1)[Ge](Cl)(Cl)C1=CC=CC=C1
C1(=CC=CC=C1)[Ge](C)(C)C1=CC=CC=C1
C1(=CC=CC=C1)[GeH2]C1=CC=CC=C1
no smiles
C(C)[Ge](Cl)(Cl)Cl
C(C)[Ge](OCC)(OCC)OCC
[GeH4]
no smiles
[Ge]
no smiles
[Ge](I)I
[Ge](=O)=O
[Ge](=[Se])=[Se]
no smiles
[Ge]=S
[Ge](Br)(Br)(Br)Br
[Ge](Cl)(Cl)(Cl)Cl
https://www.gelest.com/product-lines/germanium-compounds/?pl_page=2&perpage=50&product-code-filter=
-------------------Page2-

In [19]:
num = 5
frame = pd.DataFrame(columns=("productCode", "casNum", "compound", "smiles", "flashPoint", "img", "sds")) #frame for scraped data
url = 'https://www.gelest.com/product-lines/tin-compounds/?pl_page=1&perpage=50'
tin = scrap(url, frame, num)
tin.to_csv('tin.csv', encoding='utf-8',index=False)

-------------------Page1------------------------------------------
C(C=C)(=O)O[Sn](CCCC)(CCCC)CCCC
C(C=C)(=O)O[Sn](C1=CC=CC=C1)(C1=CC=CC=C1)C1=CC=CC=C1
C(C=C)[Sn](CCCC)(CCCC)CCCC
C(C=C)[Sn](C1=CC=CC=C1)(C1=CC=CC=C1)C1=CC=CC=C1
C[Si](C)(C)N([Si](C)(C)C)[Sn]N([Si](C)(C)C)[Si](C)(C)C
no smiles
no smiles
no smiles
no smiles
no smiles
C(CCC)[Sn](CCCC)(CCCC)C#C[Sn](CCCC)(CCCC)CCCC
no smiles
C(CCC)[Sn](CCCC)(CCCC)C=C[Sn](CCCC)(CCCC)CCCC
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
C(C)(C)(C)[Sn](CCCC)(CCCC)CCCC
C(CCC)[Sn](Cl)(Cl)Cl
no smiles
C(=O)(OC)CC[Sn](Cl)(Cl)Cl
ClC[Sn](C)(C)C
C1(C=CC=C1)[Sn](CCCC)(CCCC)CCCC
no smiles
C(C)(=O)O[Sn]OC(C)=O
C(C=C)[Sn](Br)(Br)CC=C
C(C=C)[Sn](CCCC)(CCCC)CC=C
C(CCC)[Sn](SCCCCCCCCCCCC)(SCCCCCCCCCCCC)CCCC
no smiles
no smiles
no smiles
no smiles
no smiles
C(CCC)[Sn](Cl)(OCCCC)CCCC
C(CCC)[Sn](OC(C)=O)(OC(C)=O)CCCC
no smiles
C(CCC)[Sn](OCCCC)(OCCCC)CCCC
C(CCC)[Sn](Cl)(Cl)CCCC
C(C)(C)(C)[Sn](Cl)(Cl)C(C)(C)C
htt

In [20]:
num = 3
frame = pd.DataFrame(columns=("productCode", "casNum", "compound", "smiles", "flashPoint", "img", "sds")) #frame for scraped data
url = 'https://www.gelest.com/product-lines/tin-compounds/?pl_page=1&perpage=50'
intermediate = scrap(url, frame, num)
intermediate.to_csv('specialtyPolyethersOlefins.csv', encoding='utf-8',index=False)

-------------------Page1------------------------------------------
C(C=C)(=O)O[Sn](CCCC)(CCCC)CCCC
C(C=C)(=O)O[Sn](C1=CC=CC=C1)(C1=CC=CC=C1)C1=CC=CC=C1
C(C=C)[Sn](CCCC)(CCCC)CCCC
C(C=C)[Sn](C1=CC=CC=C1)(C1=CC=CC=C1)C1=CC=CC=C1
C[Si](C)(C)N([Si](C)(C)C)[Sn]N([Si](C)(C)C)[Si](C)(C)C
no smiles
no smiles
no smiles
no smiles
no smiles
C(CCC)[Sn](CCCC)(CCCC)C#C[Sn](CCCC)(CCCC)CCCC
no smiles
C(CCC)[Sn](CCCC)(CCCC)C=C[Sn](CCCC)(CCCC)CCCC
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
C(C)(C)(C)[Sn](CCCC)(CCCC)CCCC
C(CCC)[Sn](Cl)(Cl)Cl
no smiles
C(=O)(OC)CC[Sn](Cl)(Cl)Cl
ClC[Sn](C)(C)C
C1(C=CC=C1)[Sn](CCCC)(CCCC)CCCC
no smiles
C(C)(=O)O[Sn]OC(C)=O
C(C=C)[Sn](Br)(Br)CC=C
C(C=C)[Sn](CCCC)(CCCC)CC=C
C(CCC)[Sn](SCCCCCCCCCCCC)(SCCCCCCCCCCCC)CCCC
no smiles
no smiles
no smiles
no smiles
no smiles
C(CCC)[Sn](Cl)(OCCCC)CCCC
C(CCC)[Sn](OC(C)=O)(OC(C)=O)CCCC
no smiles
C(CCC)[Sn](OCCCC)(OCCCC)CCCC
C(CCC)[Sn](Cl)(Cl)CCCC
C(C)(C)(C)[Sn](Cl)(Cl)C(C)(C)C
htt

In [ ]:
num = 5
frame = pd.DataFrame(columns=("productCode", "casNum", "compound", "smiles", "flashPoint", "img", "sds")) #frame for scraped data
url = 'https://www.gelest.com/product-lines/silicones/?pl_page=1&perpage=100'
silicon = scrap(url, frame, num)
silicon.to_csv('silicon.csv', encoding='utf-8',index=False)

-------------------Page1------------------------------------------
no smiles
no smiles
C(CCCCCCC)[Si](O[*:2])(C)[*:1]
C(CCCCCCCCCCCCC)[Si](O[*:2])(C)[*:1]
C(CCCCCCCCCCCCCCCCC)[Si](O[*:2])(C)[*:1]
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
C1(=CC=CC=C1)C(C[Si](O[*:2])(C)[*:1])C
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
C(C1CCCO1)OCCC[Si]([O])C
no smiles
C(C)[Si](O[*:2])(CC)[*:1]
C(C)[Si](O[*:2])(CC)[*:1]
C(C)[Si](O[*:2])(CC)[*:1]
C(C)[Si](O[*:2])(CC)[*:1]
C(C)[Si](O[*:2])(CC)[*:1]
C(C)[Si](O[*:2])(CC)[*:1]
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no smiles
no s